# **Evidencia de aprendizaje 1: Creación de una base de datos análiticas**

In [2]:
#Libreria
import pandas as pd
import sqlite3

In [3]:
#Cargando dataset cancer de pulmon
df=pd.read_csv("/workspaces/Bigdata_2025-2/src/bigdata/static/csv/Lung Cancer.csv")
df.head(5)

,id,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived
0,1,64.0,Male,Sweden,2016-04-05,Stage I,Yes,Passive Smoker,29.4,199,0,0,1,0,Chemotherapy,2017-09-10,0
1,2,50.0,Female,Netherlands,2023-04-20,Stage III,Yes,Passive Smoker,41.2,280,1,1,0,0,Surgery,2024-06-17,1
2,3,65.0,Female,Hungary,2023-04-05,Stage III,Yes,Former Smoker,44.0,268,1,1,0,0,Combined,2024-04-09,0
3,4,51.0,Female,Belgium,2016-02-05,Stage I,No,Passive Smoker,43.0,241,1,1,0,0,Chemotherapy,2017-04-23,0
4,5,37.0,Male,Luxembourg,2023-11-29,Stage I,No,Passive Smoker,19.7,178,0,0,0,0,Combined,2025-01-08,0


In [4]:
# Dividiendo dataframe en dos nuevos dataframes

pacientes = df[['id', 'age', 'gender', 'country', 'family_history', 'smoking_status', 'bmi']].drop_duplicates()
historial = df[['id', 'diagnosis_date', 'cancer_stage', 'cholesterol_level', 'hypertension', 'asthma', 'cirrhosis', 'other_cancer',
                     'treatment_type', 'end_treatment_date', 'survived']].copy()
historial.rename(columns={'id': 'patient_id'}, inplace=True)



In [5]:
# nuevo dataframe paciente
pacientes.head(5)

,id,age,gender,country,family_history,smoking_status,bmi
0,1,64.0,Male,Sweden,Yes,Passive Smoker,29.4
1,2,50.0,Female,Netherlands,Yes,Passive Smoker,41.2
2,3,65.0,Female,Hungary,Yes,Former Smoker,44.0
3,4,51.0,Female,Belgium,No,Passive Smoker,43.0
4,5,37.0,Male,Luxembourg,No,Passive Smoker,19.7


In [6]:
#Nuevo dataframe historial
historial.head()

,patient_id,diagnosis_date,cancer_stage,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived
0,1,2016-04-05,Stage I,199,0,0,1,0,Chemotherapy,2017-09-10,0
1,2,2023-04-20,Stage III,280,1,1,0,0,Surgery,2024-06-17,1
2,3,2023-04-05,Stage III,268,1,1,0,0,Combined,2024-04-09,0
3,4,2016-02-05,Stage I,241,1,1,0,0,Chemotherapy,2017-04-23,0
4,5,2023-11-29,Stage I,178,0,0,0,0,Combined,2025-01-08,0


In [7]:
#CReacion Cancer.db con sus respectivas tablas y carga de datos

# Conectar a SQLite
conn = sqlite3.connect("cancer_db.db")
cursor = conn.cursor()

# Creación de tablas
cursor.execute("""
CREATE TABLE IF NOT EXISTS pacientes (
    id INTEGER PRIMARY KEY,
    age INTEGER,
    gender TEXT,
    country TEXT,
    family_history TEXT,
    smoking_status TEXT,
    bmi REAL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS historial (
    record_id INTEGER PRIMARY KEY AUTOINCREMENT,
    patient_id INTEGER,
    diagnosis_date TEXT,
    cancer_stage TEXT,
    cholesterol_level REAL,
    hypertension INTEGER,
    asthma INTEGER,
    cirrhosis INTEGER,
    other_cancer TEXT,
    treatment_type TEXT,
    end_treatment_date TEXT,
    survived INTEGER,
    FOREIGN KEY(patient_id) REFERENCES patients(id)
)
""")

# Insertar datos de los DataFrames
pacientes.to_sql("pacientes", conn, if_exists="append", index=False)
historial.to_sql("historial", conn, if_exists="append", index=False)

# Verificar tablas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

conn.commit()
conn.close()

[('pacientes',), ('historial',), ('sqlite_sequence',)]
